# Dopełnienie Schur'a z użyciem eliminacji Gaussa

**Maciej Skoczeń**, **Kacper Kafara**

grupa wtorek (A) 17:50

## Środowisko obliczeniowe

**OSOBA WYKONUJĄCA OSTATECZNE OBLICZENIA POWINNA WPISAĆ TUTAJ SPECYFIKACJĘ SWOJEGO SPRZĘTU**

## Importy & typy

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import subprocess

from timeit import default_timer
from pprint import pprint
from math import sqrt

Array = np.ndarray

## Funkcje pomocnicze

In [2]:
class Timer(object):
    def __init__(self):
        self._start_time = None
        self._stop_time = None

    def start(self):
        self._start_time = default_timer()

    def stop(self):
        self._stop_time = default_timer()

    @property
    def elapsed(self, val = None):
        if self._stop_time is None or self._start_time is None:
            return None
        elapsed = self._stop_time - self._start_time
        return elapsed

# mock impl
def is_int(value) -> bool:
    as_int = int(value)
    return value == as_int

###  Wczytywanie macierzy

wygenerowanej za pomocą dostarczonego skryptu `mass_matrix`

In [3]:
def input_matrix(octave_matrix, n, m, q=1):
    result = np.zeros((n*q, m*q), dtype=np.double)
    
    for elem in octave_matrix:
        m = re.match(r"\s*\((\d+),\s*(\d+)\)\s*->\s*(\d+\.\d+)\s*", elem)
        if m is not None:
            x, y, value = m.groups()
        elif len(elem) > 0:
            coord, value = elem.strip().split(' -> ')
            value = float(value)
            x, y = coord.split(',')
            x, y = x[1:], y.strip()[:-1]
        else:
            continue
        
        for i in range(q):
            for j in range(q):
                result[i*n + int(x) - 1, j*n + int(y) - 1] = float(value)
        
    return result

In [4]:
def load_octave_matrix(filename):
    with open(filename, "r") as file:
        return file.readlines()

In [5]:
data_dir = "../../output"

def resolve_path(matrix_type, width, height = None, generate = False):
    if height is None: height = width
    path = f"{data_dir}/{matrix_type}-{width}x{height}.txt"
    if os.path.isfile(path): return path
    else: 
        if not generate:
            raise FileNotFoundError(f"Matrix file {path} not found")
            
        # do generowania macierzy potrzebny jest direnv, ustawiona zmienna 
        # środowiskowa: 
        # SCRIPT_DIR=<path-to-scripts-dir>
        # albo na sztywno ustawiona ścieżka do skryptu (ale wtedy trzeba zmodyfikować)
        # funkcję generate_matrix
        
        if width != height:
            raise ValueError("Can only generate square matrix")
            
        generate_matrix(matrix_type, width)
        
        if os.path.isfile(path): return path
        else:
            print(path)
            raise RuntimeError("Failed to generate matrix")


resolve_matrix = lambda matrix_type, n, m, q = 1: input_matrix(
    load_octave_matrix(resolve_path(matrix_type, n, m)), n, m, q
)

def resolve_matrix(matrix_type, n, m, q = 1, generate = False):
    return input_matrix(
        load_octave_matrix(resolve_path(matrix_type, n, m, generate = generate)), n, m, q
    )

def generate_matrix(matrix_type, rank):
    if matrix_type not in {'iga', 'fem'}:
        raise ValueError(f"Invalid matrix type: {matrix_type}")
        
    if rank < 16 or not is_int(sqrt(rank)):
        raise ValueError(f"Invalid matrix rank: {rank}. Must be >= 16 and sqrt(rank) must be of type integer.")
        
    rank_root = int(sqrt(rank))
    
    if matrix_type == 'fem':
        for p in range(2, 5):
            double_nxx = rank_root - p + 1
            if double_nxx % 2 == 0 and double_nxx // 2 >= 2:
                nxx = double_nxx // 2
                pxx = p
                break
        else:
            raise RuntimeError(f"Failed to determine nxx, pxx for rank: {rank}")
    else:
        for p in range(2, 5):
            nxx = rank_root - p
            if nxx >= 2:
                pxx = p
                break
        else:
            raise RuntimeError(f"Failed to determine nxx, pxx for rank: {rank}")
    
    cwd = os.getcwd()
    scripts_dir = os.getenv('SCRIPTS_DIR')
    os.chdir(scripts_dir)
    !./generate-matrix.sh cpp {matrix_type} {nxx} {pxx} 0
    os.chdir(cwd)

## Eliminacja Gaussa

In [6]:
def transform_matrix_gaussian_elim(
    A: Array,
    rows_to_transform: int,
    in_place: bool = False,
    timer: Timer = None
) -> Array:
    
    if not in_place: A = A.copy()
    
    if timer is not None:
        timer.start()

    n, _ = A.shape
    for i in range(0, min(n - 2, rows_to_transform)):
        for j in range(i + 1, n):
            factor = A[j, i] / A[i, i]
            A[j, i] = 0
            for k in range(i + 1, n):
                A[j, k] -= factor * A[i, k]
                
    if timer is not None: timer.stop()
    if not in_place: return A

## Dopełnienie Schur'a

In [7]:
def schur_complement(A: Array, complement_degree: int, timer: Timer = None) -> Array:
    transformed = transform_matrix_gaussian_elim(A,
                                                 A.shape[0] - complement_degree,
                                                 in_place = False,
                                                 timer = timer)
    return transformed[A.shape[0] - complement_degree :, A.shape[1] - complement_degree :]

In [11]:
%matplotlib

nxxs = [i for i in range(2, 31)]
pxx = 2
rxx = 0
ranks = [(nxx + pxx) ** 2 for nxx in nxxs]

matrixtype = 'iga'

main_timer = Timer()

matrices = ((resolve_matrix(matrixtype, rank, rank, generate=True), rank) for rank in ranks)
exec_times = {}

for M, rank in matrices:
    print("Computations for rank", rank)
    exec_times[rank] = []
    rank_copy = rank
    while rank_copy >= 2:
        rank_copy //= 2
        print("\trank_copy", rank_copy)
        schur_complement(M, rank_copy, timer = main_timer)
        exec_times[rank].append(main_timer.elapsed)

        
# narysować wykresy

Using matplotlib backend: TkAgg
Computations for rank 16
	rank_copy 8
	rank_copy 4
	rank_copy 2
	rank_copy 1
	rank_copy 0
Computations for rank 25
	rank_copy 12
	rank_copy 6
	rank_copy 3
	rank_copy 1
	rank_copy 0
Computations for rank 36
	rank_copy 18
	rank_copy 9
	rank_copy 4
	rank_copy 2
	rank_copy 1
	rank_copy 0
Computations for rank 49
	rank_copy 24
	rank_copy 12
	rank_copy 6
	rank_copy 3
	rank_copy 1
	rank_copy 0
Computations for rank 64
	rank_copy 32
	rank_copy 16
	rank_copy 8
	rank_copy 4
	rank_copy 2
	rank_copy 1
	rank_copy 0
Computations for rank 81
	rank_copy 40
	rank_copy 20
	rank_copy 10
	rank_copy 5
	rank_copy 2
	rank_copy 1
	rank_copy 0
Computations for rank 100
	rank_copy 50
	rank_copy 25
	rank_copy 12
	rank_copy 6
	rank_copy 3
	rank_copy 1
	rank_copy 0
Computations for rank 121
	rank_copy 60
	rank_copy 30
	rank_copy 15
	rank_copy 7
	rank_copy 3
	rank_copy 1
	rank_copy 0
Computations for rank 144
	rank_copy 72
	rank_copy 36
	rank_copy 18
	rank_copy 9
	rank_copy 4
	rank_c

In [12]:
nxxs = [i for i in range(2, 31)]
pxx = 2
rxx = 0
ranks = [(2 * nxx + pxx - 1) ** 2 for nxx in nxxs]

matrixtype = 'fem'

main_timer = Timer()

matrices = ((resolve_matrix(matrixtype, rank, rank, generate=True), rank) for rank in ranks)
exec_times = {}

for M, rank in matrices:
    print("Computations for rank", rank)
    exec_times[rank] = []
    rank_copy = rank
    while rank_copy >= 1:
        rank_copy //= 2
        print("\trank_copy", rank_copy)
        shur_complement(M, rank, timer = main_timer)
        exec_times[rank].append(main_timer.elapsed)

riga=1 (fem)
nxx=2
pxx=2
rxx=0
Computations for rank 25
	rank_copy 12
	rank_copy 6
	rank_copy 3
	rank_copy 1
	rank_copy 0
riga=1 (fem)
nxx=3
pxx=2
rxx=0
Computations for rank 49
	rank_copy 24
	rank_copy 12
	rank_copy 6
	rank_copy 3
	rank_copy 1
	rank_copy 0
riga=1 (fem)
nxx=4
pxx=2
rxx=0
Computations for rank 81
	rank_copy 40
	rank_copy 20
	rank_copy 10
	rank_copy 5
	rank_copy 2
	rank_copy 1
	rank_copy 0
riga=1 (fem)
nxx=5
pxx=2
rxx=0
Computations for rank 121
	rank_copy 60
	rank_copy 30
	rank_copy 15
	rank_copy 7
	rank_copy 3
	rank_copy 1
	rank_copy 0
riga=1 (fem)
nxx=6
pxx=2
rxx=0
Computations for rank 169
	rank_copy 84
	rank_copy 42
	rank_copy 21
	rank_copy 10
	rank_copy 5
	rank_copy 2
	rank_copy 1
	rank_copy 0
riga=1 (fem)
nxx=7
pxx=2
rxx=0
Computations for rank 225
	rank_copy 112
	rank_copy 56
	rank_copy 28
	rank_copy 14
	rank_copy 7
	rank_copy 3
	rank_copy 1
	rank_copy 0
riga=1 (fem)
nxx=8
pxx=2
rxx=0
Computations for rank 289
	rank_copy 144
	rank_copy 72
	rank_copy 36
	rank_copy